# Sensitivity Test

In [ ]:
import cartopy
import parcels
import xarray

print(f"{parcels.__version__=}")
print(f"{xarray.__version__=}")
print(f"{cartopy.__version__=}")

In [ ]:
import pooch
import xarray as xr

url = "https://github.com/LaPoGeoMar/Proj_Modelagem_Pellet/releases/download"
version = "v0.1.0"

fname = pooch.retrieve(
    url=f"{url}/{version}/model_tides_and_winds.nc",
    known_hash="sha256:1b01945c529e9f0489a659fc8360344ff58925544a2f7e543148d4f31c6dd0e8",
)

ds = xr.open_dataset(fname)
ds

In [ ]:
ds = ds.isel(
    m=slice(1, -1),
    n=slice(1, -1),
)

ds = ds.transpose("time", "n", "m")

In [ ]:
from parcels import FieldSet

variables = {
    "U": "velocity_x",
    "V": "velocity_y",
}

dimensions = {
    "U": {
        "time": "time",
        "lat": "n",  # "latitude",
        "lon": "m",  # "longitude",
    },
    "V": {
        "time": "time",
        "lat": "n",  # "latitude",
        "lon": "m",  # "longitude",
    },
}

fieldset = FieldSet.from_xarray_dataset(ds, variables, dimensions)

In [ ]:
import numpy as np

offset = 0.1  # make it away from the coast
x, y = -48.66 + offset, -26.89  # Itajaí


def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx


xi = find_nearest(ds["m"], x)
yi = find_nearest(ds["n"], y)

x = ds["m"][xi]
y = ds["n"][yi]

# Change `npart` and check the tracks to guide the amount fo particles we need to release.

In [ ]:
import random
from datetime import timedelta

from parcels import JITParticle, ParticleSet, Variable

npart = 10  # number of released particles
lon = [x] * npart  # lon de liberacao das particulas
lat = [y] * npart  # lat da liberacao das particulas

lons, lats = [], []
for k in range(npart):
    lons.append(x + random.uniform(-1, 1) * 0.01)
    lats.append(y + random.uniform(-1, 1) * 0.01)


class AgeParticle(JITParticle):
    age = Variable("age", initial=0)


pset = ParticleSet(
    fieldset=fieldset,
    pclass=AgeParticle,
    lon=lons,
    lat=lats,
)

domain = {
    "N": ds["latitude"].max().to_numpy()[()],
    "S": ds["latitude"].min().to_numpy()[()],
    "E": ds["longitude"].max().to_numpy()[()],
    "W": ds["longitude"].min().to_numpy()[()],
}

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from cartopy.feature import NaturalEarthFeature
from cartopy.mpl.gridliner import LATITUDE_FORMATTER, LONGITUDE_FORMATTER

# Coastline
feature = NaturalEarthFeature(
    name="coastline",
    category="physical",
    scale="10m",
    edgecolor="#000000",
    facecolor="#AAAAAA",
)

bbox = (
    ds["longitude"].min().to_numpy()[()],
    ds["longitude"].max().to_numpy()[()],
    ds["latitude"].min().to_numpy()[()],
    ds["latitude"].max().to_numpy()[()],
)


def creat_map(projection=ccrs.PlateCarree(), figsize=(9, 9)):
    fig, ax = plt.subplots(
        figsize=figsize,
        subplot_kw={
            "projection": projection,
        },
    )
    gl = ax.gridlines(draw_labels=True)
    gl.top_labels = gl.right_labels = False
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    ax.coastlines(resolution="10m")
    return fig, ax


# Figure
fig, ax = creat_map(projection=ccrs.PlateCarree(), figsize=(9, 9))
ax.plot(x, y, "ro", zorder=2)
ax.plot(lons, lats, "r.", zorder=1, alpha=0.65)
ax.add_feature(feature, zorder=0)
ax.set_extent(bbox)

ax.plot(
    ds["longitude"],
    ds["latitude"],
    color="blue",
    marker="o",
    markerfacecolor="none",
    alpha=0.15,
    zorder=0,
)
X, Y = np.meshgrid(ds["m"], ds["n"])
ax.add_feature(feature, zorder=1)
ax.plot(x, y, "ro")  # Itajaí
ax.plot(X, Y, "k.", alpha=0.25, zorder=0, color="lightgrey");

In [ ]:
from parcels import AdvectionRK4, StatusCode


def KeepInDomain(particle, fieldset, time):
    # https://github.com/euroargodev/VirtualFleet/blob/4e524f24e15c5dfc6b8b4f57836953b2ccc9eafe/virtualargofleet/virtualargofleet.py
    # out of geographical area : here we can delete the particle
    if particle.state == StatusCode.ErrorOutOfBounds:
        particle.delete()


def Age(particle, fieldset, time):
    # Create a custom kernel which keeps track of the particle age (minutes)
    particle.age += particle.dt / 3600


output_file = pset.ParticleFile(
    name="results-model_tides_and_winds.zarr", outputdt=timedelta(hours=12)
)

kernels = [AdvectionRK4, KeepInDomain, Age]

pset.execute(
    kernels,
    # Reduce as much as possible to increase the npart but also see particles reach the island
    runtime=timedelta(days=7),
    dt=timedelta(hours=6),
    output_file=output_file,
)

In [ ]:
from pathlib import Path


def normalize_speed(u, v):
    u_norm = u / np.sqrt(u**2.0 + v**2.0)
    v_norm = v / np.sqrt(u**2.0 + v**2.0)
    speed = (u**2 + v**2) ** 0.5
    return (u_norm, v_norm, speed)


fname = Path("avg-model_tides_and_winds.nc")
if not fname.exists():
    avg = ds.mean(dim="time")
    avg.to_netcdf(fname)
else:
    avg = xr.load_dataset(fname)

u = avg["velocity_x"].squeeze()
v = avg["velocity_y"].squeeze()
u_norm, v_norm, speed = normalize_speed(u, v)

In [ ]:
fig, ax = creat_map()
ax.contourf(avg["longitude"], avg["latitude"], speed)
ax.quiver(
    avg["longitude"],
    avg["latitude"],
    u_norm,
    v_norm,
    color="white",
    scale=50,
)

ax.plot(x, y, "bo", label="Itajaí")

for p in pset:
    ax.plot(p.lon, p.lat, "r.")

## Plot all trajectories

In [ ]:
import warnings

import trajan  # noqa
import xarray as xr

warnings.simplefilter("ignore")

ds = xr.open_zarr("results-model_tides_and_winds.zarr")

fig, ax = creat_map()
ds.traj.plot(ax=ax)
ax.add_feature(feature, zorder=99)
ax.set_extent(bbox)

# Distance travelled

TODO